In [1]:
import torch
from perceiver_pytorch import PerceiverIO

In [9]:
model = PerceiverIO(
    dim = 1,                    # dimension of sequence to be encoded
    queries_dim = 3,            # dimension of decoder queries
    logits_dim = 8192,            # dimension of final logits
    depth = 6,                   # depth of net
    num_latents = 256,           # number of latents, or induced set points, or centroids. different papers giving it different names
    latent_dim = 512,            # latent dimension
    cross_heads = 1,             # number of heads for cross attention. paper said 1
    latent_heads = 8,            # number of heads for latent self attention, 8
    cross_dim_head = 64,         # number of dimensions per cross attention head
    latent_dim_head = 64,        # number of dimensions per latent self attention head
    weight_tie_layers = False    # whether to weight tie layers (optional, as indicated in the diagram)
)

seq = torch.randn(1, 64 * 64, 1)
queries = torch.randn(1, 64 * 64, 3)
mask = torch.ones(1, 64 * 64).bool()

logits = model(seq, queries=queries, mask=mask)
logits.shape

torch.Size([1, 4096, 8192])